In [11]:
import pandas as pd
from scripts.pyensembl_operations import import_pyensembl
import sqlite3
pd.set_option('display.max_columns', None)   # Show all columns


# Initialize pyensembl
g37 = import_pyensembl(37)

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


In [4]:
df = pd.read_csv("results/streamlined/results_0.45_sig421.csv")
df = df[df["p_adj"] < 0.05]
df.head()

,gene_id,upregulated_real,downregulated_real,upregulated_synth,downregulated_synth,log2_odds_ratio,shrunk_log2_odds,p_value,p_adj,z_score
4,ENSG00000000460,541,101,402.3,140.9,0.905200,0.905200,0.000019,0.005046,0.540616
17,ENSG00000001630,91,25,54.7,51.3,1.751558,1.751558,0.000034,0.007526,1.188706
37,ENSG00000003400,50,8,48.9,38.0,2.211096,2.211095,0.000115,0.016915,1.540592
51,ENSG00000004487,10,26,76.4,35.9,-2.414648,-2.414647,0.000028,0.006707,-2.001521
230,ENSG00000009307,20,43,60.9,36.4,-1.820009,-1.820009,0.000183,0.022326,-1.546183


In [8]:
with sqlite3.connect('data/mirscribe.db') as sqlite_conn:
    genes = pd.read_sql('SELECT * FROM genes', sqlite_conn)
    mirnas = pd.read_sql('SELECT * FROM mirnas', sqlite_conn)

# add genes table details
cols_to_merge_genes = ['gene_id','gene_name', 'is_oncogene_oncokb', 'is_tsupp_oncokb',
       'is_brca_driver', 'tier_cosmic', 'is_hallmark_cosmic',
       'is_tsupp_cosmic', 'is_oncogene_cosmic', 'is_oncogene_consensus',
       'is_tsupp_consensus', 'cancer_gene_role', "is_brca_driver"]

df = pd.merge(df, genes[cols_to_merge_genes], how="left", on="gene_id" )

In [12]:
df.head()

,gene_id,upregulated_real,downregulated_real,upregulated_synth,downregulated_synth,log2_odds_ratio,shrunk_log2_odds,p_value,p_adj,z_score,gene_name,is_oncogene_oncokb,is_tsupp_oncokb,is_brca_driver,tier_cosmic,is_hallmark_cosmic,is_tsupp_cosmic,is_oncogene_cosmic,is_oncogene_consensus,is_tsupp_consensus,cancer_gene_role,is_brca_driver
0,ENSG00000000460,541,101,402.3,140.9,0.905200,0.905200,0.000019,0.005046,0.540616,C1orf112,0,0,0,0,0,0,0,0,0,neither,0
1,ENSG00000001630,91,25,54.7,51.3,1.751558,1.751558,0.000034,0.007526,1.188706,CYP51A1,0,0,0,0,0,0,0,0,0,neither,0
2,ENSG00000003400,50,8,48.9,38.0,2.211096,2.211095,0.000115,0.016915,1.540592,CASP10,0,0,0,0,0,0,0,0,0,neither,0
3,ENSG00000004487,10,26,76.4,35.9,-2.414648,-2.414647,0.000028,0.006707,-2.001521,KDM1A,0,0,0,0,0,0,0,0,0,neither,0
4,ENSG00000009307,20,43,60.9,36.4,-1.820009,-1.820009,0.000183,0.022326,-1.546183,CSDE1,0,0,0,0,0,0,0,0,0,neither,0


In [14]:
df.sort_values(by="shrunk_log2_odds")

,gene_id,upregulated_real,downregulated_real,upregulated_synth,downregulated_synth,log2_odds_ratio,shrunk_log2_odds,p_value,p_adj,z_score,gene_name,is_oncogene_oncokb,is_tsupp_oncokb,is_brca_driver,tier_cosmic,is_hallmark_cosmic,is_tsupp_cosmic,is_oncogene_cosmic,is_oncogene_consensus,is_tsupp_consensus,cancer_gene_role,is_brca_driver
344,ENSG00000227449,0,26,95.0,29.2,-7.412958,-7.412956,3.785422e-14,3.774571e-10,-5.828922,RP11-111F5.5,0,0,0,0,0,0,0,0,0,neither,0
354,ENSG00000230011,0,15,24.3,8.7,-6.384831,-6.330701,6.523461e-07,6.098213e-04,-5.041645,CTSLP4,0,0,0,0,0,0,0,0,0,neither,0
274,ENSG00000182487,1,22,63.9,12.0,-6.272023,-6.271696,2.407159e-12,1.200129e-08,-4.955264,NCF1B,0,0,0,0,0,0,0,0,0,neither,0
371,ENSG00000233132,0,5,26.2,3.0,-6.390845,-6.175489,2.012535e-04,2.343370e-02,-5.046250,FAM90A3P,0,0,0,0,0,0,0,0,0,neither,0
407,ENSG00000258279,0,5,39.1,6.0,-6.066420,-6.024668,2.180521e-04,2.482943e-02,-4.797826,LINC00592,0,0,0,0,0,0,0,0,0,neither,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,ENSG00000162980,70,0,55.3,17.3,5.491163,5.491160,5.326005e-06,2.333673e-03,4.052267,ARL5A,0,0,0,0,0,0,0,0,0,neither,0
64,ENSG00000111981,24,0,16.2,15.5,5.552934,5.532508,3.047527e-05,7.066956e-03,4.099567,ULBP1,0,0,0,0,0,0,0,0,0,neither,0
111,ENSG00000125875,60,0,31.0,12.1,5.596935,5.596755,1.003089e-05,3.615231e-03,4.133261,TBC1D20,0,0,0,0,0,0,0,0,0,neither,0
397,ENSG00000249635,59,0,28.4,11.9,5.674088,5.673821,1.495879e-05,4.552465e-03,4.192340,RP11-710F7.3,0,0,0,0,0,0,0,0,0,neither,0
